In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-03"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1714,2024-08-03,Chile Lnb 2,20:45,Stadio Italiano,Truenos De Talca,2.95,1.33,138.5,1.85,1.81,0.0,0.00,0.00,bFzrFjmA,0.338983,0.751880,0.540541,0.552486,0.090863,138.538,82.964388,0.598857,1.8,1.643168,0.912871,276.66,2.238,1.475862,0.659456,-9.0,98.132,19.534025,0.199059,3.0,0.000000,0.000000,74.58,1.304,0.206228,0.158150,57.0,58,66,4.77,1.13,95.522,114.338,0.535286,0.015456,NaN,-0.15,-0.030,-65.000000,0.000000,0.0,0.000000,1.42,0.284,1.161972,0.000000,0.0,0.000000
1715,2024-08-03,Chile Lnb 2,21:30,Sergio Ceppi,Colo Colo,3.19,1.32,145.5,1.83,1.83,0.0,0.00,0.00,4GsZGCIc,0.313480,0.757576,0.546448,0.546448,0.071055,302.400,279.008577,0.922647,1.8,1.643168,0.912871,585.00,4.332,4.392092,1.013872,25.0,88.284,22.512496,0.255001,3.0,0.000000,0.000000,94.64,1.078,0.091761,0.085121,89.0,78,91,7.50,1.04,101.280,103.610,0.586381,0.000000,NaN,0.22,0.044,49.772727,0.000000,0.0,0.000000,1.62,0.324,0.987654,0.000000,0.0,0.000000
1716,2024-08-03,Indonésia Ibl,09:00,Pelita Jaya,Satria Muda,2.04,1.66,161.5,1.83,1.83,1.5,1.93,1.52,YBw8ahJr,0.490196,0.602410,0.546448,0.546448,0.092606,102.850,14.962911,0.145483,3.0,0.000000,0.000000,121.50,1.156,0.159937,0.138354,87.0,125.276,30.986908,0.247349,2.4,1.341641,0.559017,173.88,1.310,0.335857,0.256379,83.0,90,92,1.35,1.89,100.966,149.922,0.145244,0.000000,0.168066,0.82,0.164,6.341463,0.867107,0.9,0.032893,-0.63,-0.126,-5.238095,0.772276,0.6,-0.172276
1717,2024-08-03,República Dominicana Lnb,20:30,Marineros,Caneros,1.64,2.06,169.5,1.83,1.83,0.0,0.00,0.00,p6XRVE5t,0.609756,0.485437,0.546448,0.546448,0.095193,185.798,71.997163,0.387502,1.2,1.643168,1.369306,152.00,2.224,0.762122,0.342681,-18.0,155.858,34.922334,0.224065,2.4,1.341641,0.559017,145.29,1.876,0.545096,0.290563,-4.0,80,87,1.90,1.67,0.000,0.000,0.160532,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1718,2024-08-03,República Dominicana Lnb,21:00,Titanes Del Licey,Metros de Santiago,1.59,2.16,176.5,1.86,1.80,0.0,0.00,0.00,IJVZThzg,0.628931,0.462963,0.537634,0.555556,0.091894,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,243.178,60.675784,0.249512,1.8,1.643168,0.912871,199.50,2.572,0.686491,0.266909,13.0,111,95,1.68,2.10,0.000,0.000,0.214960,0.023184,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1719,2024-08-03,República Dominicana Lnb,21:30,Reales de la Vega,Indios,1.72,1.95,176.5,1.83,1.83,-2.5,1.99,2.37,EoPrSW45,0.581395,0.512821,0.546448,0.546448,0.094216,122.586,23.435091,0.191173,1.8,1.643168,0.912871,163.80,1.438,0.227310,0.158074,9.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,90,77,1.82,1.97,0.000,0.000,0.088629,0.000000,0.123257,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1720,2024-08-03,Vietnã Vba,09:30,Hanoi Buffaloes,Saigon Heat,2.65,1.45,160.5,1.86,1.80,0.0,0.00,0.00,IquR34tH,0.377358,0.689655,0.537634,0.555556,0.067014,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,85,75,1.95,1.20,0.000,0.000,0.413916,0.023184,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1721,2024-08-03,Bolívia Libobasquet,21:00,Tarija Basquet,Pichi

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
1716,09:00,Indonésia Ibl,Pelita Jaya,Satria Muda,2.04,Back Home
1747,04:30,Austrália Nbl1 Sul,Keilor Thunder F,Ballarat F,2.19,Back Home
1750,03:00,Austrália Nbl1 Leste,Norths Bears F,Sutherland F,2.38,Back Home
1751,21:05,Porto Rico Bsn,Leones De Ponce,Criollos de Caguas,1.61,Back Home
